In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql.functions import when, udf, col, regexp_replace, regexp_extract
from pyspark.sql.types import DoubleType,IntegerType, StringType 

In [2]:
sc = SparkContext( 'local' )  
sqlCtx = SQLContext( sc )

In [4]:
emp = [ ("Lisa", "Sales", 10000, 35),
        ("Evan", "Sales", 32000, 38),
        ("Fred", "Engineering", 21000, 28),
        ("Alex", "Sales", 30000, 33),
        ("Tom", "Engineering", 23000, 33),
        ("Jane", "Marketing", 29000, 28),
        ("Jeff", "Marketing", 35000, 38),
        ("Paul", "Engineering", 29000, 23),
        ("Chloe", "Engineering", 23000, 25) ]

In [5]:
empDF = sqlCtx.createDataFrame( emp,('name' , 'dept' , 'salary' , 'age' ))
empDF.show()

+-----+-----------+------+---+
| name|       dept|salary|age|
+-----+-----------+------+---+
| Lisa|      Sales| 10000| 35|
| Evan|      Sales| 32000| 38|
| Fred|Engineering| 21000| 28|
| Alex|      Sales| 30000| 33|
|  Tom|Engineering| 23000| 33|
| Jane|  Marketing| 29000| 28|
| Jeff|  Marketing| 35000| 38|
| Paul|Engineering| 29000| 23|
|Chloe|Engineering| 23000| 25|
+-----+-----------+------+---+



In [6]:
empDF.createOrReplaceTempView('employees') 

In [ ]:
## 부서별로 급여가 많은 순위를 구하시오.(partition)

In [7]:
sql='''SELECT name, dept,
        RANK() OVER (PARTITION BY dept ORDER BY salary) AS rank FROM employees
    '''
sqlCtx.sql( sql ).show() 

+-----+-----------+----+
| name|       dept|rank|
+-----+-----------+----+
| Lisa|      Sales|   1|
| Alex|      Sales|   2|
| Evan|      Sales|   3|
| Fred|Engineering|   1|
|  Tom|Engineering|   2|
|Chloe|Engineering|   2|
| Paul|Engineering|   4|
| Jane|  Marketing|   1|
| Jeff|  Marketing|   2|
+-----+-----------+----+



In [8]:
sql='''SELECT max(salary) FROM employees
    '''
sqlCtx.sql( sql ).show() 

+-----------+
|max(salary)|
+-----------+
|      35000|
+-----------+



In [9]:
empDF.selectExpr('max(salary)').show()

+-----------+
|max(salary)|
+-----------+
|      35000|
+-----------+



In [12]:
empDF.selectExpr('name','dept', 'RANK() OVER (PARTITION BY dept ORDER BY salary) AS rank' ).show()

+-----+-----------+----+
| name|       dept|rank|
+-----+-----------+----+
| Lisa|      Sales|   1|
| Alex|      Sales|   2|
| Evan|      Sales|   3|
| Fred|Engineering|   1|
|  Tom|Engineering|   2|
|Chloe|Engineering|   2|
| Paul|Engineering|   4|
| Jane|  Marketing|   1|
| Jeff|  Marketing|   2|
+-----+-----------+----+



In [15]:
sql='''SELECT name, dept,
         min(salary) OVER (PARTITION BY dept ORDER BY salary) AS min FROM employees
    '''
sqlCtx.sql( sql ).show() 

+-----+-----------+-----+
| name|       dept|  min|
+-----+-----------+-----+
| Lisa|      Sales|10000|
| Alex|      Sales|10000|
| Evan|      Sales|10000|
| Fred|Engineering|21000|
|  Tom|Engineering|21000|
|Chloe|Engineering|21000|
| Paul|Engineering|21000|
| Jane|  Marketing|29000|
| Jeff|  Marketing|29000|
+-----+-----------+-----+

